In [2]:
import pandas as pd
import bs4 as bs
import urllib3
import tqdm

In [36]:
#match_url = "https://www.kicker.de/freiburg-gegen-heidenheim-2025-bundesliga-4936969/schema"

columns_list = ["Match_Index", "Team_Home", "Team_Away", "Minute", "Goals_Home", "Goals_Away", "Subs_Home", "Subs_Away",
        "Event", "Event_Team","Current_Points","Final_Points", "Event_Coach"]

season_columns_list = ["Season","Match_Day","Match_Index", "Team_Home", "Team_Away", "Goals_Home", "Goals_Away", "Home_Coach", "Away_coach"]

match_event_df = pd.DataFrame()
matches_df = pd.DataFrame()

base_url = "https://www.kicker.de/bundesliga/spieltag/"


In [ ]:
def get_season_events(season_events_df, season_matches_df, start_year, start_md=0, end_md=34):
    season = str(start_year)+"-"+str((start_year+1))[-2:].rjust(2,"0")
    season_url = base_url+season+"/"
    for i in tqdm(range(start_md,end_md+1)):
        md_url = season_url+str(i)+"/"

        http = urllib3.PoolManager()
        resp = http.request("GET", md_url)

        kicker_Soup = bs.BeautifulSoup(resp.data, "lxml")
        resp.close()

        match_container = kicker_Soup.findAll("a",{"class":"kick__v100-scoreBoard kick__v100-scoreBoard--standard"})

        for k in range(len(match_container)):
            link = match_container[k]["href"]
            link = "https://www.kicker.de"+str(link).replace("analyse","schema")

            if "schema" in link:
                season_events_df, season_matches_df = get_match_events(season_events_df, season_matches_df, link, season, i, columns_list, season_columns_list)
    
    return season_events_df, season_matches_df

def get_match_events(all_df, season_df, url,season, match_day,columns, season_columns):
    match_index = url.split("-").pop().split("/")[0]
    
    #abgebrochene spiele filtern
    if match_index in ("4721736","4936909","807810","1013490"):
        return all_df
    
    http = urllib3.PoolManager()
    resp = http.request("GET", url)

    match_Soup = bs.BeautifulSoup(resp.data, "lxml")
    resp.close()

    final_goals = match_Soup.findAll("div", {"kick__v100-scoreBoard__scoreHolder__score"})
    final_home_goals = int(final_goals[0].text)
    final_away_goals = int(final_goals[1].text)
                
    coach_container = match_Soup.find("section",{"class":"kick__section-item kick__section-item--sec"})
    
    coaches = coach_container.findAll("div",{"class":"kick__lineup-text"})
    home = coaches[0].text.split("\n")
    home = list(filter(None, home))
    home_team = home[0]
    home_coach = home[1]
    home_goals = 0
    home_subs = 0
    away = coaches[1].text.split("\n")
    away = list(filter(None, away))
    away_team = away[0]
    away_coach = away[1]
    away_goals = 0
    away_subs = 0
    current_points = 1

    final_home_points = final_away_points = 1
    if final_home_goals > final_away_goals:
        final_home_points = 3
        final_away_points = 0
    if final_away_goals > final_home_goals:
        final_home_points = 0
        final_away_points = 3

    event_container = match_Soup.find("div", {"class":"kick__game-timeline"})
    events = event_container.findAll("div",{"class":"kick__ticker-event-overlay-box kick__js_overlay-card kick__ticker-event-overlay-box--with-player"})

    match_events = []

    for event in events:
        text = event.text
        goals = text.count("Tor") - text.count("Tor annulliert")
        subs = text.count("Spielerwechsel")
        reds = text.count("Rote Karte")
        yellows = text.count("Gelbe Karte")
        minute = text[8:].split(".")[0]
        if home_goals == away_goals:
            current_points = 1
        if home_team in text:
            if home_goals > away_goals:
                current_points = 3
            if home_goals < away_goals:
                current_points = 0
            for goal in range(goals):
                home_goals += 1
                match_events.append((match_index,home_team,away_team,minute,home_goals,away_goals,home_subs,away_subs,"Tor",home_team, current_points,final_home_points,home_coach))
            for sub in range(subs):
                home_subs += 1
                match_events.append((match_index,home_team,away_team,minute,home_goals,away_goals,home_subs,away_subs,"Wechsel",home_team,current_points,final_home_points,home_coach))
            for red in range(reds):
                match_events.append((match_index,home_team,away_team,minute,home_goals,away_goals,home_subs,away_subs,"Rote Karte",home_team,current_points,final_home_points,home_coach))
            for yellow in range(yellows):
                match_events.append((match_index,home_team,away_team,minute,home_goals,away_goals,home_subs,away_subs,"Gelbe Karte",home_team,current_points,final_home_points,home_coach))
        else:
            if home_goals < away_goals:
                current_points = 3
            if home_goals > away_goals:
                current_points = 0
            for goal in range(goals):
                away_goals += 1
                match_events.append((match_index,home_team,away_team,minute,home_goals,away_goals,home_subs,away_subs,"Tor",away_team,current_points,final_away_points,away_coach))
            for sub in range(subs):
                away_subs += 1
                match_events.append((match_index,home_team,away_team,minute,home_goals,away_goals,home_subs,away_subs,"Wechsel",away_team,current_points,final_away_points,away_coach))
            for red in range(reds):
                match_events.append((match_index,home_team,away_team,minute,home_goals,away_goals,home_subs,away_subs,"Rote Karte",away_team,current_points,final_away_points,away_coach))
            for yellow in range(yellows):
                match_events.append((match_index,home_team,away_team,minute,home_goals,away_goals,home_subs,away_subs,"Gelbe Karte",away_team,current_points,final_away_points,away_coach))

    df = pd.DataFrame(data = match_events,columns=columns)
    df["Event_ID"] = df["Match_Index"]+"_"+df.index.astype(str).str.pad(3,"left","0")

    all_df = pd.concat([all_df,df])

    match_info = [[match_index, season, match_day, home_team, away_team, final_home_goals, final_away_goals, home_coach, away_coach]]
    match_df = pd.DataFrame(data = match_info, columns=season_columns)

    season_df = pd.concat([season_df, match_df])

    return all_df, season_df

In [44]:
for s in [2022,2023]:
    match_event_df, matches_df = get_season_events(match_event_df, matches_df, s)

In [45]:
match_event_df

,Match_Index,Team_Home,Team_Away,Minute,Goals_Home,Goals_Away,Subs_Home,Subs_Away,Event,Event_Team,Current_Points,Final_Points,Event_Coach,Event_ID
0,4781432,Frankfurt,Bayern,5,0,1,0,0,Tor,Bayern,1,3,Nagelsmann,4781432_000
1,4781432,Frankfurt,Bayern,11,0,2,0,0,Tor,Bayern,3,3,Nagelsmann,4781432_001
2,4781432,Frankfurt,Bayern,29,0,3,0,0,Tor,Bayern,3,3,Nagelsmann,4781432_002
3,4781432,Frankfurt,Bayern,35,0,4,0,0,Tor,Bayern,3,3,Nagelsmann,4781432_003
4,4781432,Frankfurt,Bayern,43,0,5,0,0,Tor,Bayern,3,3,Nagelsmann,4781432_004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,4862271,Heidenheim,Köln,78,3,1,1,5,Gelbe Karte,Köln,0,0,Schultz,4862271_013
14,4862271,Heidenheim,Köln,78,4,1,1,5,Tor,Heidenheim,3,3,Raab,4862271_014
15,4862271,Heidenheim,Köln,79,4,1,2,5,Wechsel,Heidenheim,3,3,Raab,4862271_015
16,4862271,Heidenheim,Köln,86,4,1,3,5,Wechsel,Heidenheim,3,3,Raab,4862271_016
